# Figure 4 Analysis: Phenotype Complexity vs Performance

This notebook analyzes how ELDER and ontology-only approaches perform across different phenotype complexity levels.

## Setup

First, let's import the required libraries and set up our analysis environment.

In [ ]:
import json
import sys
from pathlib import Path
import matplotlib.pyplot as plt
import numpy as np
import duckdb

# Set up paths
DATA_DIR = Path("../data")
RESULTS_DIR = Path("../results")
RESULTS_DIR.mkdir(exist_ok=True)

# Configuration
DEFAULT_DATASET = "normalized_phenopackets_07"  # Best ELDER performance
DB_PATH = DATA_DIR / "Best_Match_Cosine_all_combined_Elder_vs_Exomiser.db"

print(f"Data directory: {DATA_DIR}")
print(f"Results directory: {RESULTS_DIR}")
print(f"Default dataset: {DEFAULT_DATASET}")

## Check Available Datasets

Let's see what normalized datasets are available:

In [ ]:
# List available datasets
available_datasets = []
for item in DATA_DIR.iterdir():
    if item.is_dir() and item.name.startswith("normalized_phenopackets"):
        file_count = len(list(item.glob("*.json")))
        available_datasets.append((item.name, file_count))

print("Available normalized datasets:")
for dataset, count in sorted(available_datasets):
    marker = " ⭐" if dataset == DEFAULT_DATASET else ""
    print(f"  {dataset}: {count} files{marker}")

print(f"\n⭐ {DEFAULT_DATASET} is the default (best ELDER performance)")

## Define Analysis Functions

Let's define the core analysis functions:

In [ ]:
def load_dataset_and_bin_by_complexity(dataset_path):
    """
    Load phenopackets and assign them to complexity bins.
    
    Returns:
        - bins: Dictionary mapping bin names to lists of filenames
        - file_to_disease: Dictionary mapping filenames to disease IDs
    """
    bins = {
        "1-5": [],
        "10-15": [],
        "15-20": [],
        "20-25": [],
        "25-30": [],
        "30-35": [],
        "35-40": [],
        "40-45": [],
        "45+": [],
    }
    
    file_to_disease = {}
    phenotype_counts = []
    
    print(f"Loading dataset from: {dataset_path}")
    
    for json_file in dataset_path.glob("*.json"):
        try:
            with open(json_file, 'r') as f:
                phenopacket = json.load(f)
            
            # Extract disease ID
            if 'diseases' in phenopacket and phenopacket['diseases']:
                disease_id = phenopacket['diseases'][0]['term']['id']
                file_to_disease[json_file.name] = disease_id
            
            # Count phenotypes
            phenotype_count = 0
            if 'phenotypicFeatures' in phenopacket:
                phenotype_count = len(phenopacket['phenotypicFeatures'])
            
            phenotype_counts.append(phenotype_count)
            
            # Assign to bins
            filename = json_file.name
            if 1 <= phenotype_count <= 5:
                bins["1-5"].append(filename)
            elif 10 <= phenotype_count <= 15:
                bins["10-15"].append(filename)
            elif 16 <= phenotype_count <= 20:
                bins["15-20"].append(filename)
            elif 21 <= phenotype_count <= 25:
                bins["20-25"].append(filename)
            elif 26 <= phenotype_count <= 30:
                bins["25-30"].append(filename)
            elif 31 <= phenotype_count <= 35:
                bins["30-35"].append(filename)
            elif 36 <= phenotype_count <= 40:
                bins["35-40"].append(filename)
            elif 41 <= phenotype_count <= 45:
                bins["40-45"].append(filename)
            elif phenotype_count > 45:
                bins["45+"].append(filename)
        
        except Exception as e:
            print(f"Error processing {json_file}: {e}")
            continue
    
    # Print bin summary
    total_binned = sum(len(files) for files in bins.values())
    print(f"\nDataset summary:")
    print(f"  Total files processed: {len(phenotype_counts)}")
    print(f"  Files assigned to bins: {total_binned}")
    print(f"  Average phenotypes per case: {np.mean(phenotype_counts):.1f}")
    
    print(f"\nBin distribution:")
    for bin_name, files in bins.items():
        print(f"  {bin_name}: {len(files)} files")
    
    return bins, file_to_disease

In [ ]:
def query_performance_data(bins, db_path):
    """
    Query the database for ELDER and Exomiser performance data.
    
    Returns:
        - elder_per_case: Dictionary mapping filenames to ELDER top-1 accuracy
        - exomiser_per_case: Dictionary mapping filenames to Exomiser top-1 accuracy
    """
    elder_per_case = {}
    exomiser_per_case = {}
    
    print(f"\nQuerying database: {db_path}")
    
    if not db_path.exists():
        print(f"❌ Error: Database not found at {db_path}")
        print("Please download the database from the Zenodo link provided in README.md")
        return elder_per_case, exomiser_per_case
    
    conn = duckdb.connect(str(db_path))
    
    # Get all filenames from all bins
    all_files = []
    for bin_files in bins.values():
        all_files.extend(bin_files)
    
    print(f"Querying performance data for {len(all_files)} files...")
    
    successful_queries = 0
    
    for filename in all_files:
        query = """
        SELECT Exomiser, cosBMA_ELDER_large3
        FROM Exomiser_vs_cosBMA_ELDER_large3_disease_rank_comparison
        WHERE phenopacket = ?
        """
        
        try:
            result = conn.execute(query, [filename]).fetchone()
            if result:
                exomiser_rank, elder_rank = result
                # Convert rank to top-1 accuracy (1 if rank=1, 0 otherwise)
                exomiser_top1 = 1 if exomiser_rank == 1 else 0
                elder_top1 = 1 if elder_rank == 1 else 0
                
                exomiser_per_case[filename] = exomiser_top1
                elder_per_case[filename] = elder_top1
                successful_queries += 1
            else:
                exomiser_per_case[filename] = 0
                elder_per_case[filename] = 0
        
        except Exception as e:
            print(f"Database query error for {filename}: {e}")
            exomiser_per_case[filename] = 0
            elder_per_case[filename] = 0
    
    conn.close()
    
    print(f"✅ Successfully queried {successful_queries}/{len(all_files)} files")
    
    return elder_per_case, exomiser_per_case

In [ ]:
def calculate_bin_accuracies(bins, elder_per_case, exomiser_per_case):
    """
    Calculate average top-1 accuracy for each phenotype complexity bin.
    
    Returns:
        - elder_bin_accuracies: Dictionary mapping bin names to ELDER accuracies
        - exomiser_bin_accuracies: Dictionary mapping bin names to Exomiser accuracies
    """
    elder_bin_accuracies = {}
    exomiser_bin_accuracies = {}
    
    print(f"\nCalculating bin-wise accuracies:")
    
    for bin_name, files in bins.items():
        if files:
            elder_accuracies = [elder_per_case.get(f, 0) for f in files]
            exomiser_accuracies = [exomiser_per_case.get(f, 0) for f in files]
            
            elder_bin_accuracies[bin_name] = sum(elder_accuracies) / len(elder_accuracies)
            exomiser_bin_accuracies[bin_name] = sum(exomiser_accuracies) / len(exomiser_accuracies)
            
            print(f"  Bin {bin_name}: {len(files)} files")
            print(f"    ELDER: {elder_bin_accuracies[bin_name]:.3f}")
            print(f"    Ontology-only: {exomiser_bin_accuracies[bin_name]:.3f}")
            print(f"    Difference: {elder_bin_accuracies[bin_name] - exomiser_bin_accuracies[bin_name]:+.3f}")
        else:
            elder_bin_accuracies[bin_name] = 0
            exomiser_bin_accuracies[bin_name] = 0
    
    return elder_bin_accuracies, exomiser_bin_accuracies

In [ ]:
def create_figure(elder_bin_accuracies, exomiser_bin_accuracies, dataset_name, save_plots=True):
    """
    Create the comparative bar chart (Figure 4).
    
    Parameters:
        - elder_bin_accuracies: ELDER accuracies by bin
        - exomiser_bin_accuracies: Exomiser accuracies by bin
        - dataset_name: Name of dataset being analyzed
        - save_plots: Whether to save the plots to files
    """
    plt.figure(figsize=(14, 8))
    
    bin_names = list(elder_bin_accuracies.keys())
    elder_accuracies = list(elder_bin_accuracies.values())
    exomiser_accuracies = list(exomiser_bin_accuracies.values())
    
    x = np.arange(len(bin_names))
    width = 0.35
    
    # Create bars
    bars1 = plt.bar(x - width/2, exomiser_accuracies, width, label='Ontology-only', 
                   color='lightcoral', edgecolor='black', alpha=0.7)
    bars2 = plt.bar(x + width/2, elder_accuracies, width, label='ELDER', 
                   color='skyblue', edgecolor='black', alpha=0.7)
    
    # Formatting
    plt.xlabel('Phenotype Count Bins', fontsize=12)
    plt.ylabel('Average Top-1 Accuracy', fontsize=12)
    plt.ylim(0, 1.0)
    plt.xticks(x, bin_names)
    plt.legend(fontsize=11)
    
    # Add value labels on bars
    for bar, accuracy in zip(bars1, exomiser_accuracies):
        plt.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.01,
                f'{accuracy:.3f}', ha='center', va='bottom', fontweight='bold', fontsize=9)
    
    for bar, accuracy in zip(bars2, elder_accuracies):
        plt.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.01,
                f'{accuracy:.3f}', ha='center', va='bottom', fontweight='bold', fontsize=9)
    
    plt.grid(True, alpha=0.3, axis='y')
    plt.tight_layout()
    
    # Add subtitle with dataset info
    plt.figtext(0.5, 0.02, f'Dataset: {dataset_name}', ha='center', fontsize=10, style='italic')
    
    if save_plots:
        # Save plots
        svg_path = RESULTS_DIR / 'figure_4_phenotype_complexity.svg'
        png_path = RESULTS_DIR / 'figure_4_phenotype_complexity.png'
        
        plt.savefig(svg_path, bbox_inches='tight')
        plt.savefig(png_path, dpi=300, bbox_inches='tight')
        
        print(f"\n💾 Plots saved:")
        print(f"  SVG: {svg_path}")
        print(f"  PNG: {png_path}")
    
    plt.show()
    
    # Calculate overall performance
    elder_overall = np.mean(elder_accuracies)
    exomiser_overall = np.mean(exomiser_accuracies)
    difference = elder_overall - exomiser_overall
    
    print(f"\n📊 Overall Performance Summary:")
    print(f"  ELDER average: {elder_overall:.3f}")
    print(f"  Ontology-only average: {exomiser_overall:.3f}")
    print(f"  Difference: {difference:+.3f} ({'ELDER better' if difference > 0 else 'Ontology-only better'})")

## Run the Analysis

Now let's run the complete analysis pipeline:

In [ ]:
# Select dataset to analyze
DATASET_TO_ANALYZE = DEFAULT_DATASET  # Change this to use a different dataset
dataset_path = DATA_DIR / DATASET_TO_ANALYZE

print(f"🔬 Running Figure 4 Analysis")
print(f"📁 Dataset: {DATASET_TO_ANALYZE}")
print("=" * 50)

# Step 1: Load dataset and bin by complexity
bins, file_to_disease = load_dataset_and_bin_by_complexity(dataset_path)

In [ ]:
# Step 2: Query performance data from database
elder_per_case, exomiser_per_case = query_performance_data(bins, DB_PATH)

In [ ]:
# Step 3: Calculate bin-wise accuracies
elder_bin_accuracies, exomiser_bin_accuracies = calculate_bin_accuracies(
    bins, elder_per_case, exomiser_per_case
)

In [ ]:
# Step 4: Create and display Figure 4
create_figure(elder_bin_accuracies, exomiser_bin_accuracies, DATASET_TO_ANALYZE, save_plots=True)

## Compare Different Datasets (Optional)

You can run this cell to quickly compare how different datasets perform:

In [ ]:
# Compare multiple datasets (optional)
COMPARE_DATASETS = True  # Set to False to skip this comparison

if COMPARE_DATASETS and len(available_datasets) > 1:
    print("🔄 Comparing multiple datasets...\n")
    
    dataset_comparisons = []
    
    # Analyze first 3 datasets for comparison
    for dataset_name, _ in available_datasets[:3]:
        print(f"Analyzing {dataset_name}...")
        
        dataset_path = DATA_DIR / dataset_name
        bins_temp, _ = load_dataset_and_bin_by_complexity(dataset_path)
        elder_temp, exomiser_temp = query_performance_data(bins_temp, DB_PATH)
        elder_acc_temp, exomiser_acc_temp = calculate_bin_accuracies(
            bins_temp, elder_temp, exomiser_temp
        )
        
        # Calculate overall performance
        elder_overall = np.mean(list(elder_acc_temp.values()))
        exomiser_overall = np.mean(list(exomiser_acc_temp.values()))
        difference = elder_overall - exomiser_overall
        
        dataset_comparisons.append({
            'dataset': dataset_name,
            'elder_accuracy': elder_overall,
            'exomiser_accuracy': exomiser_overall,
            'difference': difference
        })
        
        print(f"  ELDER: {elder_overall:.3f}, Ontology-only: {exomiser_overall:.3f}, Diff: {difference:+.3f}\n")
    
    # Sort by difference
    dataset_comparisons.sort(key=lambda x: x['difference'], reverse=True)
    
    print("📊 Dataset Comparison Summary:")
    for i, result in enumerate(dataset_comparisons, 1):
        marker = " ⭐" if result['dataset'] == DEFAULT_DATASET else ""
        print(f"{i}. {result['dataset']}: ELDER advantage = {result['difference']:+.3f}{marker}")
else:
    print("Skipping dataset comparison...")

## Summary

This notebook has generated Figure 4, which shows:

1. **Performance comparison** between ELDER and ontology-only approaches
2. **Phenotype complexity analysis** across 9 different complexity bins
3. **Top-1 accuracy metrics** for both approaches

### Key Outputs:
- `figure_4_phenotype_complexity.svg` - Vector format for publication
- `figure_4_phenotype_complexity.png` - Raster format for presentations
- Console output with detailed accuracy statistics

### Next Steps:
- Review the generated figure in the `results/` directory
- Use the SVG version for publication-quality figures
- Analyze the performance patterns across complexity bins

---
*This analysis is part of the ELDER project. For more information, see the README.md file.*